# 1B5 Baseline Run

Without any experimental tweaks

In [28]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-selective-loss"
DEEPSPEED_STRAT="deepspeed_stage_1"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# The model sizing
MODEL_NAME="RWKV-v5-1B5-world.bin"
MODEL_URL="https://huggingface.co/BlinkDL/rwkv-5-world/resolve/main/RWKV-5-World-1B5-v2-20231025-ctx4096.pth?download=true"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v5-exp/selective-loss
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [29]:
# Lets wget the model files
!cd "{PROJECT_DIR}" && mkdir -p "{PROJECT_DIR}/model"
!cd "{PROJECT_DIR}/model" && \
    wget -O "{MODEL_NAME}" -nc "{MODEL_URL}"

File ‘RWKV-v5-1B5-world.bin’ already there; not retrieving.


In [34]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/1B5-sloss-enwiki_10k-world-packing.yaml"

Saving the dataset (1/1 shards): 100%|█| 646/646 [00:00<00:00, 7625.90 examples/
Saving the dataset (1/1 shards): 100%|█| 803/803 [00:00<00:00, 33706.89 examples


# Multi-epoch training

In [37]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/1B5-sloss-enwiki_10k-world-packing.yaml" \
        --model.load_model="../model/{MODEL_NAME}" \
        --data.skip_datapath_setup=True \
        --trainer.callbacks.init_args.dirpath="../checkpoint/selective-loss/baseline/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - 1B5 - Baseline (packsize=16k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.target_batch_size=128 \
        --trainer.microbatch_size=8 \
        --model.ctx_len=1024 \
        --trainer.devices="{GPU_DEVICES}"

[2024-01-21 05:28:21,163] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v5-exp/selective-loss/config/1B5-enwiki_10k-world-packing.yaml', '--model.load_model=../model/RWKV-v5-1B5-world.bin', '--data.skip_datapath_setup=True', '--trainer.callbacks.init_args.dirpath=../checkpoint/selective-loss/baseline/', '--trainer.logger.init_args.name=infctx-v5-selective-loss - 1B5 - Baseline (packsize=16k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.target_batch_size=128', '--